In [161]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn
rcParams['figure.figsize'] = 20, 12


In [162]:
import h2o
    
# Start H2O on your local machine
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 28 mins
H2O cluster version:,3.10.4.7
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_albertomariopirovano_6zbq57
H2O cluster total nodes:,1
H2O cluster free memory:,1.570 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [167]:
X_train = h2o.import_file("../chapter_9/X_train.csv")
X_test = h2o.import_file("../chapter_9/X_test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [175]:
X_train = X_train.drop([0])
X_test = X_test.drop([0])
X_train = X_train[1:,:]
X_test = X_test[1:,:]

In [176]:
from matplotlib import cm

def plot_bidimensional(model, test, recon_error, layer, title):

    bidimensional_data = model.deepfeatures(test, layer).cbind(recon_error).as_data_frame()

    cmap = cm.get_cmap('Spectral')

    fig, ax = plt.subplots()
    bidimensional_data.plot(kind='scatter', 
                            x='DF.L{}.C1'.format(layer+1), 
                            y='DF.L{}.C2'.format(layer+1), 
                            s = 500,
                            c = 'Reconstruction.MSE',
                            title = title,
                            ax = ax,
                            colormap=cmap)
    layer_column = 'DF.L{}.C'.format(layer + 1)
    columns = [layer_column + '1', layer_column + '2']
    for k, v in bidimensional_data[columns].iterrows():
        ax.annotate(k, v, size=20, verticalalignment='bottom', horizontalalignment='left')
    fig.canvas.draw()


In [178]:
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator
seed = 13

model = H2OAutoEncoderEstimator(
    activation="Tanh",
    hidden=[50, 20, 2, 20, 50],
    epochs=100,
    #sparse=True,
    #l1=1e-5,
    seed=seed,
    reproducible=True)

In [ ]:
model.train(
    x=X_train.names,
    training_frame=X_train
)

recon_error = model.anomaly(X_test)
plot_bidimensional(model, X_test, recon_error, 2, "2D representation of data points seed {}".format(seed))

deeplearning Model Build progress: |█████████████████████

In [ ]:
model

In [ ]:
plt.figure()
df=recon_error.as_data_frame(True)
df["sample_index"]=df.index
df.plot(kind="scatter", x="sample_index", y="Reconstruction.MSE", 
        title = "reconstruction error", s = 500)

In [ ]:
len(recon_error)

In [ ]:
model.deepfeatures(X_train, 1).as_data_frame().plot(kind='scatter', x='DF.L2.C1', y='DF.L2.C2')

Anomaly detection using compressed representation

In [87]:
for seed in range(1, 6):
    model = H2OAutoEncoderEstimator(
        activation="Tanh",
        hidden=[50,20, 2, 20, 50],
        epochs=100,
        #sparse=True,
        #l1=1e-5,
        seed=seed,
        reproducible=True)
    model.train(
    x=X_train.names,
    training_frame=X_train)

    recon_error = model.anomaly(X_test)
    plot_bidimensional(model, X_train, recon_error, 2, "2D representation of data points seed {}".format(seed))
    # compute average and variance of the 2 dimensions


deeplearning Model Build progress: |████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_b2f69b858c52d0f22f6a722e59ee4d2f> was cancelled by the user.

In [133]:
model = H2OAutoEncoderEstimator(
        activation="Tanh",
        hidden=[50,20, 2, 20, 50],
        epochs=100,
        #sparse=True,
        #l1=1e-5,
        seed=1,
        reproducible=True)
model.train(
    x=train_ecg.names,
    training_frame=train_ecg
)

recon_error = model.anomaly(test_ecg)
bidimensional_data = model.deepfeatures(test_ecg, 2).cbind(recon_error).as_data_frame()
bidimensional_data

deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%
deepfeatures progress: |██████████████████████████████████████████████████████████████████████| 100%


,DF.L3.C1,DF.L3.C2,Reconstruction.MSE
0,0.487688,0.863620,0.006362
1,0.635581,0.756164,0.011617
2,0.764004,0.777907,0.008139
3,0.847438,0.879615,0.024230
4,0.653252,0.918426,0.005973
5,0.033087,0.895865,0.014068
6,-0.849062,0.803780,0.021128
7,-0.934323,0.278897,0.008563
8,-0.953484,0.554313,0.015090
9,-0.954942,0.565842,0.016041
